// Cross Correlation and Template Matching TEMPLATE_IMAGE = 'animal-family-25-template.jpg' SOURCE = 'animal-family-25.jpg'

In [108]:
# CODE HERE:
import matplotlib.pyplot as plt 
import matplotlib.image as img
import numpy as np
import cv2

In [109]:
# READING IN IMAGES:
template_img = cv2.imread('./ASSETS/animal-family-25-template.jpg', cv2.IMREAD_GRAYSCALE).astype(float)
template = ((template_img - np.min(template_img)) * (1.0/(np.max(template_img) - np.min(template_img)) * 255.0))



source_img = cv2.imread('./ASSETS/animal-family-25.jpg', cv2.IMREAD_GRAYSCALE).astype(float)
source = ((source_img - np.min(source_img)) * (1.0/(np.max(source_img) - np.min(source_img)) * 255.0))


In [110]:
# FUNCTIONS/HELPER FUNCTIONS:
def cross_correlation(image, template):
    # Get the dimensions of the image and the template
    image_height = len(image)
    image_width = len(image[0])
    template_height = len(template)
    template_width = len(template[0])

    # Calculate the padding sizes
    pad_height = template_height // 2
    pad_width = template_width // 2

    # Pad the image with zeros on all sides
    padded_image = np.pad(image, [(pad_height, pad_height), (pad_width, pad_width)], mode='constant', constant_values=0)

    # Calculate the zero-mean versions of the image and the template
    template_mean = template.mean()
    zero_mean_template = template - template_mean

    # Create an output array with the same size as the image
    output = np.zeros_like(image)

    # Perform cross-correlation
    for i in range(pad_height, pad_height + image_height):
        for j in range(pad_width, pad_width + image_width):
            # Extract the current region of interest of the image
            region = padded_image[i - pad_height:i + pad_height + 1, j - pad_width:j + pad_width + 1]
            # Subtract mean of the current region of interest
            current_roi_mean = region.mean()
            zero_mean_roi = region - current_roi_mean
            # Calculate correlation score
            correlation_score = np.sum(zero_mean_roi * zero_mean_template)
            # Store score
            output[i - pad_height, j - pad_width] = correlation_score


    return output

def overlay_images(base_image, overlay_image):
    # Get dimensions of the images
    height = len(base_image)
    width = len(base_image[0])

    # Create output image of the same size
    output_image = [[0 for _ in range(width)] for _ in range(height)]

    # Alpha value for blending (0.5 for equal blending of both images)
    alpha = 0.5

    # Loop over each pixel and blend the base image and overlay image
    for i in range(height):
        for j in range(width):
            base_pixel = base_image[i][j]
            overlay_pixel = overlay_image[i][j]
            # Blend pixels together using the alpha value
            blended_pixel = int((1 - alpha) * base_pixel + alpha * overlay_pixel)
            output_image[i][j] = blended_pixel

    return output_image

def manual_threshold(img_in, threshold):
    
    manual_thresh_img = []
    
    for row in img_in:
        thresholded_row = []
        for pixel in row:
            thresholded_row.append(255 if pixel > threshold else 0)
        manual_thresh_img.append(thresholded_row)

    threshold_img = manual_thresh_img

    return threshold_img

def imageAdjust(image_array):
    # Calculate the minimum and maximum pixel values of the image
    min_val = np.min(image_array)
    max_val = np.max(image_array)
    
    # Adjust the image array to the range [0, 255]
    adjusted_image_array = 255 * (image_array - min_val) / (max_val - min_val)
    
    # Ensure the values are within the byte range and of type uint8
    adjusted_image_array = adjusted_image_array.astype(np.uint8)
    
    return adjusted_image_array


In [118]:
# RESULTS:
correlation_result = cross_correlation(source, template)
binarized_peaks = manual_threshold(imageAdjust(correlation_result), 180)
overlay_image = overlay_images(source, binarized_peaks)

# WRITING RESULTS:
cv2.imwrite('correlation_result.png', imageAdjust(correlation_result))
cv2.imwrite('binarized_peaks.png', np.array(binarized_peaks))
cv2.imwrite('overlay_image.png', np.array(overlay_image))

True